In [1]:
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import tensorflow
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
tf.enable_eager_execution()

In [16]:
base_dir = "/home/debjani/Desktop/droughtwatch1/data/train"
get_files_path = base_dir + "/*"
training_files = sorted(glob.glob(get_files_path))
#print((training_files))

def parse_visual(data):
    #count = 0
    dataset = tf.data.TFRecordDataset(data)
    # pattern for one part file
    # dataset = tf.data.TFRecordDataset('part-r-00099')
    iterator = dataset.make_one_shot_iterator()
    features = {
      'B1': tf.io.FixedLenFeature([], tf.string),
      'B2': tf.io.FixedLenFeature([], tf.string),
      'B3': tf.io.FixedLenFeature([], tf.string),
      'B4': tf.io.FixedLenFeature([], tf.string),
      'B5': tf.io.FixedLenFeature([], tf.string),
      'B5': tf.io.FixedLenFeature([], tf.string),
      'B6': tf.io.FixedLenFeature([], tf.string),
      'B7': tf.io.FixedLenFeature([], tf.string),
      'B8': tf.io.FixedLenFeature([], tf.string),
      'B9': tf.io.FixedLenFeature([], tf.string),
      'B10': tf.io.FixedLenFeature([], tf.string),
      'B11': tf.io.FixedLenFeature([], tf.string),
      'label': tf.io.FixedLenFeature([], tf.int64),
    }  

    parsed_examples = [tf.parse_single_example(data, features) for data in iterator]
    #count+=1
    #print(count)
    return parsed_examples


In [17]:
def get_img_from_example(parsed_example, intensify=True):
    bandArray = np.zeros((65,65,11), 'uint8')
    for i, band in enumerate(['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9','B10', "B11"]):
        band_data = np.frombuffer(parsed_example[band].numpy(), dtype=np.uint8)
        band_data = band_data.reshape(65, 65)
        if intensify:
            band_data = band_data/np.max(band_data)*255
        else:
            band_data = band_data*255
        bandArray[..., i] = band_data
        
    label = tf.cast(parsed_example['label'], tf.int32).numpy()
        
    return bandArray, label

In [18]:
img_idx = 0
img_list = []
label_list= []
parsed_examples = []
#print(training_files[1])
for i in range(len(training_files) - 1):
    parsed_examples = parse_visual(training_files[i]) 
    for j in range(0,(len(parsed_examples)-1)):
        image, label = get_img_from_example(parsed_examples[j])
        image_dir = os.path.join(base_dir, 'images_all')
        label_dir = os.path.join(base_dir, 'label_all')
        if not os.path.exists(image_dir):
            os.mkdir(image_dir)
        filename = image_dir + "/" + "image_" + str(img_idx).zfill(5)
        np.save(filename, image)
        
        if not os.path.exists(label_dir):
            os.mkdir(label_dir)
        label_list.append(str(img_idx).zfill(5) + " " + str(label))
        #label_list.append(label)
        img_idx += 1
file = label_dir + "/" +  "train_labels.txt"
with open(file, "w") as f:
    for s in label_list:
        #print(s)
        f.write(str(s) +"\n")


/home/debjani/anaconda3/envs/env3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys
